# Workshop GeoUtrecht - Reverse Fault - 21.08.2020

**Authors: Alexander Jüstel (CGRE - RWTH Aachen University), Arthur Endlein Correira**

# Overview

`GemGIS` is a Python-based, **open-source geographic information processing library**. It is capable of preprocessing spatial data such as vector data (shape files, geojson files, geopackages), raster data, data obtained from WMS services or XML/KML files. Preprocessed data can be stored in a dedicated Data Class to be passed to the geomodeling package [GemPy](https://github.com/cgre-aachen/gempy) in order to accelerate to model building process. Postprocessing of model results will allow export from `GemPy` to geoinformation systems such as QGIS and ArcGIS or to Google Earth for further use. 

`GemGIS` uses the full functionality of [GeoPandas](https://geopandas.org/), [rasterio](https://rasterio.readthedocs.io/en/latest/#), [OWSLib](https://geopython.github.io/OWSLib/), [Pandas](https://pandas.pydata.org/) and [NumPy](https://numpy.org/).

All provided maps and examples were taken from the books 'Interpretation of Geological Structures Through Maps: An Introductory Practical Manual' by D. Powell and "An Introduction to Geological Structures and Maps" by G.M. Bennison referenced at the bottom. referenced at the bottom.

# Motivation

`GemGIS` was build upon the idea of Prof. Florian Wellmann (CGRE - RWTH Aachen University) to have a tighter integration of spatial data with `GemPy`. Larger models (Master Thesis Alexander Jüstel) made it necessary to think about workflows to simplify the procedures to get a larger number of data into `GemPy`. 

***What is the aim of GemGIS?***
- Accelerate processing of spatial data
- Enhance post-modeling functionalities
- Create a tool for teaching purposes
- Interactively creating GemPy Models

# Installation
Due to rasterio, GemGIS must be used with **python==3.7**

1) `conda install geopandas`

2) `conda install rasterio`

3) `pip install gemgis`


# Structure of Package

The core of `GemGIS` is made of the `GemPyData` class (`gemgis.py`). Its attributes can directly be utilized by `GemPy` making it easier for users to load data. Methods of the `GemPyData` class allow users to directly set these attributes. Multiple other files contain functions to manipulate vector data, raster data, etc.:

* `gemgis.py` - core file containing the `GemPyData` class
* `vector.py` - file containing functions to manipulate vector data
* `raster.py` - file containing functions to manipulate raster data
* `utils.py` - file containing utility functions frequently used for the manipulation of vector/raster data
* `wms.py` - file containing methods to load WMS services as arrays/rasters
* `visualization.py` - file containing functions to simplify plotting of spatial data
* `postprocessing.py` - file containing functions to postprocess GemPy geo_model data
* `notebooks` - folder containing tutorial notebooks explaining the features of `GemGIS` and example notebooks applying these features

# Features

<a name="vector"></a>
### Extracting Data from Vector Files

Data stored as points, lines or polygons as shape-files, geopackages or geojson files can easily be imported into `GemGIS` GeoPandas GeoDataFrames. X and Y coordinates can then be extracted for these objects for direct use in `GemPy`. Digital elevations models can be interpolated if contour lines with height values are provided. If the loaded the exceeds the desired modeling/working are extent, the data can be cropped. 

<a name="raster"></a>
### Extracting Data from Raster Files

Rasters (stored as arrays in Python) such as digital elevation models store height information. The height of interface points can be extracted from these rasters. In addition, if a raster represents a layer in the subsurface, orientation values can be sampled for the use in `GemPy`. Orientations are calculated via the slope and aspect of the provided raster. It is also possible to resize rasters, clip rasters or save rasters as referenced geotiffs again. 

<a name="wms"></a>
### Extracting Data from Online Services (in development)

Online services provide a wide range of possibilities to work with spatial data. Currently, it is possible to load data from WMS services into `GemGIS`. The functionality will be extended to WCS and WFS services in the future.

<a name="xml/kml"></a>
### Extracting Data from XML/KML Files (in development)
XML/KML Data export will be available in the future. 

<a name="pyvista"></a>
### Visualization of Data in PyVista
`PyVista` is the main 3D visualization package of `GemPy`. In order for new users to get used to the package, it is possible to plot the input data as a `PyVista` plot. 

<a name="utils"></a>
### Utility Tools
`GemGIS` offers a wide range of utility tools. These includes 
* Conversion of vector data into custom sections directly usable in `GemPy`
* Conversion of GeoDataFrames into Pandas DataFrames for `GemPy`
* Setting the extent and resolution for a `GemPy` model based on vector data and lists
* Load and save QGIS style files (QML) for the use as color_dict in `GemPy`
* Calculate orientations based on strike lines
* Interpolate missing strike lines to calculate orientations
* Read CSV files as GeoDataFrames
* and many more to come


<a name="post"></a>
### Postprocessing of GemPy geo_model data
`GemGIS` also offers postprocessing methods to use the data of the `GemPy` model. These include:
* Export of the geological map as shape file
* Extract boreholes from `GemPy` models
* Create depth maps of surfaces
* and many more to come

# Workshop GeoUtrecht

The image below shows an outcropping coal seam with layers above and below the coal seam as well as a fault striking
roughly W-E. The area is  763 m wide (W-E extent) and 989 m high (N-S extent). The scale is neglected.

Tasks: 
- Create a `GemPy` Model to visualize the spatial distribution of the coal seam. 

- Determine the type of fault (normal, reverse, transform)

- Where would you recommend to drill (taking the type of fault into account)?


<img src="../data/task.png" width="300">

# Data Audit

***What is available:***
- Contour Lines
- Layer Boundaries
- Fault Trace

***What is needed:***
- Interface points
- Orientation values

***How do we get the missing data?***
-> via strike lines

***What do we need to do now?***
- Digitize contour lines
- Digitize layer boundaries
- Create strike lines
- Create custom section

# Importing Libraries

In [2]:
import os
# Add your user name in ****
os.environ['PROJ_LIB'] = r'C:\Users\****\miniconda3\envs\gu\Library\share\proj'

In [3]:
import sys
sys.path.append('../gemgis')
import gemgis as gg
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Data Preaparation

## Load Basemap and Geological Map

In [4]:
base_map = rasterio.open('../data/task.png')
geological_map = gpd.read_file('../data/geolmap.geojson')

C:\Users\legui\miniconda3\envs\gu2\lib\site-packages\rasterio\__init__.py:219: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


## Digitize and Load Contour Data

## Plot Topography

In [ ]:
extent = gpd.read_file('../data/extent.geojson')
extent = gg.utils.set_extent(gdf=extent, minz=100, maxz=900)

# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
topo.plot(ax=ax2, column = 'Z', legend = False, linewidth = 5, cmap = 'gist_earth',
          aspect='equal')

# orientations.plot(ax=ax2, column = 'formation', legend = False, s = 300)

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.grid()
ax2.set_ylim(extent[2],extent[3])
ax2.set_xlim(extent[0],extent[1]);

## Interpolate Topography

In [ ]:
dem = gg.vector.interpolate_raster(topo, method='rbf')
dem

In [ ]:
np.save('../dem.npy',dem)

In [ ]:
dem = np.load('../dem.npy')

## Plot Topography

In [ ]:
extent = gpd.read_file('data/extent.geojson')
extent = gg.utils.set_extent(gdf=extent, minz=100, maxz=900)

# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
im = plt.imshow(dem,origin = 'lower', alpha=0.75, cmap='gist_earth')
cbar = plt.colorbar(im)
cbar.set_label('m')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_ylim(extent[2],extent[3])
ax2.set_xlim(extent[0],extent[1]);


## Save Topography as Raster

In [ ]:
gg.raster.save_as_tiff('../topo.tif',dem, crs='EPSG:4326', extent=[0,765,0,1000])

## Digitize and Load Layer Boundaries

## Extract coordinates

## Plot Interfaces

In [ ]:
extent = gpd.read_file('../data/extent.geojson')
extent = gg.utils.set_extent(gdf=extent, minz=100, maxz=900)

# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
interfaces.plot(ax=ax2, column = 'formation', legend = True, linewidth = 10,
                aspect='equal')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_ylim(extent[2],extent[3])
ax2.set_xlim(extent[0],extent[1]);


## Calculate Orientations

<img src="../data/orientations.jpg" width="500">
<img src="../data/task.png" width="300">

## Digitizing and Loading Fault Strike

In [ ]:
# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
strike_fault.plot(ax=ax2, column = 'formation', legend = False, linewidth = 5,
                  aspect='equal')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1]);

Based on the strike lines and the calculated angles, we can conclude that the fault is dipping towards the south. Please take into account that the map was not digitized to scale. So angles are usually higher than expected for certain types of faults!

***What is the dip direction of fault?***

## Digitize and Load Interface Strike
For the strike of the different layers we have to distinguish between layer 1 and layer 2 and between the two fault blocks. 

In [ ]:
# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
strike_layers.plot(ax=ax2, column = 'formation', legend = True, linewidth = 5,
                   aspect='equal')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1]);

## Separate Layers

In [ ]:
gdf1 = strike_layers[strike_layers['formation'] == 'Layer1']
gdf2 = strike_layers[strike_layers['formation'] == 'Layer2']
gdf1_south = gdf1[gdf1['id']<= 4]
gdf1_north = gdf1[gdf1['id']> 4]
gdf2_south = gdf2[gdf2['id']<= 5]
gdf2_north = gdf2[gdf2['id']> 5]

## Calculate Orientations

In [ ]:
orientations1_south = gg.utils.calculate_orientations(gdf1_south)
orientations1_north = gg.utils.calculate_orientations(gdf1_north)
orientations2_south = gg.utils.calculate_orientations(gdf2_south)
orientations2_north = gg.utils.calculate_orientations(gdf2_north)
orientations2_north

## Merge DataFrames

In [ ]:
orientations_coords = pd.concat([orientations_fault, orientations1_south, orientations1_north, orientations2_south, orientations2_north]).reset_index()
orientations_coords

In [ ]:
# Creating a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(20,10))
# Plotting the geological map
ax1.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1])

# Plotting the geological map
ax2.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
# Plotting the different elements on the geological map
ax2.scatter(orientations_coords['X'], orientations_coords['Y'])
strike_layers.plot(ax=ax2, column = 'formation', legend = True, linewidth = 5, aspect='equal')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.grid()
ax1.set_ylim(extent[2],extent[3])
ax1.set_xlim(extent[0],extent[1]);

***What direction are the layers dipping?***
***Which one is the oldest? Which one is the youngest?***

# Let us combine everything - Creating a GemPy Data Class

## Set Extent

## Set Resolution

## Set Interfaces

## Add Section Dict

## Add Surface Colors

## Add Stack

## Add DEM

# Visualize Data Set

In [ ]:
import pyvista as pv
p = pv.Plotter(notebook =True)
gg.visualization.plot_dem_3d(dem, p, cmap = 'gist_earth')
gg.visualization.plot_contours_3d(topo, p, color = 'red', add_to_z = 10)
gg.visualization.plot_points_3d(interfaces_coords, p, color = 'blue', add_to_z = 10)
gg.visualization.plot_points_3d(orientations_coords, p, color = 'orange', add_to_z = 20)
gg.visualization.plot_contours_3d(interfaces_coords, p, color = 'blue', add_to_z = 10)

p.camera_position =[(-283.285811675846, -1597.1397046051004, 1155.542325449192), 
                    (577.9371599370799, 495.3480261506809, 381.7124055285182), 
                    (0.17313457304419916, 0.27814381639313923, 0.9448070898437746)]
p.set_background('white')
p.show_grid(color='black')
p.show()

# Create GemPy Model

## Importing GemPy

Please see https://docs.gempy.org/installation.html for more information on how to install GemPy.

In [ ]:
import gempy as gp
print(gp)
print(gp.__version__)

## Creating GemPy Model

For more information on how to create a GemPy Model, please see the tutorials at: https://docs.gempy.org/tutorials/index.html. With the attributes of the GemPy Data Class, all necessary variables can be passed to the model.

In [ ]:
gp.plot_2d(geo_model, direction = 'z')
plt.grid()

A raster created with ArcGIS has to be loaded as the import of the created raster fails. This is due to a bug in `GemPy`described here: https://github.com/cgre-aachen/gempy/issues/492

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    verbose=[],
                    update_kriging = False
                    )

The geological map had to be created manually as there seems to be a bug when creating the geological map with `GemGIS`. This was reported already: https://github.com/cgre-aachen/gempy/issues/446

In [ ]:
shape = geo_model._grid.topography.values_2d[:, :, 2].shape
geolmap = geo_model.solutions.geological_map[0].reshape(shape)
cols = ['#069a2b', '#b35a2a','#525252', ]
plt.figure(figsize=(10,10))
plt.imshow(np.flipud(base_map.read(1)), origin = 'lower', cmap ='gray')
gmap = np.rot90(geolmap,1)
plt.imshow(gmap, extent=geo_data.extent[:4], alpha=0.75, cmap=ListedColormap(cols))

# Postprocessing - Converting Geological Map to Shape Files

## Create GeoDataFrame with Polygons

## Plot Polygons

## Save Polygons as Shape Files

These polygons can now be saved as shape files and displayed for example as in QGIS shown below. 

## Save Polygons as GeoTiff

Rasters like DEMs can be saved as georeferenced tif-files. The same can be done with the geological map obtained from `GemPy` by saving the array of the geological map as tif. 

In [ ]:
gg.raster.save_as_tiff('../geolmap.tif',gmap, extent=geo_data.extent, crs='EPSG:4326')

# Create borehole from Geo_Model

Geological models are used to extract information from the subsurface. As geologists, we like to look of the result of the model at a single location and down to the maximum z extent in depth. This is termed a borehole or in the case of fluid extraction a well. These boreholes can easily be extracted and displayed using GemGIS. All you need is the `geo_model` object, the `geo_data` object and the location of your borehole, optionally with its maximum depth. 

# Depth Maps

Depth maps indicate the depth of a surface within the model extent. By applying an intuitive color coding, the spatial position of the respective surface can easily be interpreted. Depth maps can easily be created by executing `gg.visualization.plot_depth_map(...)`. All that is needed is the `geo_model` object and the name of the surface to be plotted. Optionally, a range for the colorbar can be provided to make the maps of different surfaces comparable. Otherwise, the min and max values of each surface are taken for the limits of the color bar. By setting `notebook` to `False` an interactive PyVista Window is opened (press 'Q' to close window safely again).

## Depth Map Layer1

## Depth Map Layer2